In [1]:
import torch

from project.models.TwoTower.SequenceEncoder import SequenceEncoder as SE

B, L, Emb = 2, 5, 32
genre_k = 3
model = SE(item_count = 100, genre_count = 10, emb_dim = Emb, max_seq_len = L)
model.eval()
history_seq = torch.LongTensor([
    [10, 20, 30, 3, 2],
    [0, 0, 3, 4, 1]
])
genre_sep = torch.randint(0, 10, (B, L, genre_k)) # 确保 padding 位置的 genre 也是 0 (虽然你的 embedding 会处理，但为了严谨)
genre_sep[history_seq == 0] = 0

print("输入形状构造完毕: History", history_seq.shape)


输入形状构造完毕: History torch.Size([2, 5])


In [ ]:
import pandas as pd

train = pd.read_pickle("./data/cleaned/train_set.pkl")

genre0list = train['hist_genre_ids'].head(1).tolist()
movie0list = train['hist_movie_ids'].head(1).tolist()
print(genre0list)
print(movie0list)



[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [3]:
import numpy as np
def check_data_health(df, cols_to_check):
    print(f"--- 正在检查列: {cols_to_check} ---")
    
    for col in cols_to_check:
        series = df[col]
        
        # 1. 检查 NaN
        nan_count = series.isna().sum()
        
        # 2. 检查 Inf (无穷大)
        # np.isinf 对字符串或对象类型会报错，先确保是数值
        if pd.api.types.is_numeric_dtype(series):
            inf_count = np.isinf(series).sum()
        else:
            inf_count = "无法检查(非数值类型)"
            
        # 3. 检查是否出现了负数 (因为 log1p 输入不能小于 -1)
        neg_count = (series < 0).sum() if pd.api.types.is_numeric_dtype(series) else "N/A"
        
        print(f"列名: {col:<20} | NaN数: {nan_count} | Inf数: {inf_count} | 负数: {neg_count} | 类型: {series.dtype}")
        
        if nan_count > 0 or (isinstance(inf_count, int) and inf_count > 0):
            print(f"🚨 警告: {col} 这一列有问题！")
            # 打印出有问题的行看看
            print(df[df[col].isna() | (np.isinf(df[col]) if isinstance(inf_count, int) else False)].head())

# 运行检查
cols = ['user_activity_log', 'movie_pop_log', 'movie_avg_rate_log']
check_data_health(train, cols)


--- 正在检查列: ['user_activity_log', 'movie_pop_log', 'movie_avg_rate_log'] ---
列名: user_activity_log    | NaN数: 0 | Inf数: 0 | 负数: 0 | 类型: float64
列名: movie_pop_log        | NaN数: 0 | Inf数: 0 | 负数: 0 | 类型: float64
列名: movie_avg_rate_log   | NaN数: 0 | Inf数: 0 | 负数: 0 | 类型: float64
